In [35]:
import os
import numpy as np
import cv2 as cv
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from PIL import Image

In [36]:
# Mendefinisikan direktori dataset asli dan dataset hasil deteksi tepi
dataset_awal = "Dataset/"
dataset_thinning = "dataset_thinning"

# Memeriksa apakah folder dataset hasil sudah ada, jika tidak buat folder baru
if not os.path.exists(dataset_thinning):
    os.makedirs(dataset_thinning)

# Mendapatkan daftar nama file gambar dalam dataset awal
daftar_file = os.listdir(dataset_awal)

In [37]:
print(daftar_file)

['Sedan', 'Sport', 'SUV']


In [38]:
# Fungsi gray
def Grayscale (image):
    ker_gauss = np.ones((3,3))/9
    image0 = image[:,:,0]
    image1 = image[:,:,1]
    image2 = image[:,:,2]
    filteredimage0=cv.filter2D(image0,-1,ker_gauss)
    filteredimage1=cv.filter2D(image1,-1,ker_gauss)
    filteredimage2=cv.filter2D(image2,-1,ker_gauss)
    return filteredimage0/3 + filteredimage1/3 + filteredimage2/3

In [39]:
# Fungsi Resize
def resize (image):
    res = cv.resize(image, (150, 150), interpolation=cv.INTER_NEAREST)
    return res

In [40]:
# Fungsi Threshold
def Threshold(image):
    matriks = np.array(image)
    result = np.zeros(matriks.shape).astype(int)
    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            if matriks[i,j] <= 128 : 
                result[i,j] = 255
            else:
                result[i,j] = 0

    return result

In [41]:
def thinning(image):
    skeleton = np.zeros(image.shape, dtype=np.uint8)
    prev = np.zeros(image.shape, dtype=np.uint8)
    diff = None

    while True:
        prev[:] = skeleton
        for step in range(1, 3):
            for i in range(1, image.shape[0] - 1):
                for j in range(1, image.shape[1] - 1):
                    P2 = image[i-1, j]
                    P3 = image[i-1, j+1]
                    P4 = image[i, j+1]
                    P5 = image[i+1, j+1]
                    P6 = image[i+1, j]
                    P7 = image[i+1, j-1]
                    P8 = image[i, j-1]
                    P9 = image[i-1, j-1]
                    A = (P2 == 0 and P3 == 1) + (P3 == 0 and P4 == 1) + \
                        (P4 == 0 and P5 == 1) + (P5 == 0 and P6 == 1) + \
                        (P6 == 0 and P7 == 1) + (P7 == 0 and P8 == 1) + \
                        (P8 == 0 and P9 == 1) + (P9 == 0 and P2 == 1)
                    B = P2 + P3 + P4 + P5 + P6 + P7 + P8 + P9
                    m1 = step == 1 and (P2 * P4 * P6 == 0) and (P4 * P6 * P8 == 0)
                    m2 = step == 2 and (P2 * P4 * P8 == 0) and (P2 * P6 * P8 == 0)
                    if A == 1 and 2 <= B <= 6 and m1 and m2:
                        skeleton[i, j] = 0
                    else:
                        skeleton[i, j] = image[i, j]
        diff = np.sum(np.abs(skeleton - prev))
        if diff == 0:
            break
    return skeleton

In [42]:
# Fungsi Normalisasi
def normalisasi(image):
    min_val = np.min(image)
    max_val = np.max(image)
    
    normalized_image = (image - min_val) / (max_val - min_val)
    uint8_image = (normalized_image * 255).astype(np.uint8)
    
    return uint8_image

In [43]:
# for nama_file in daftar_file:

#     # Membaca gambar dari dataset awal
#     gambar_awal = cv.imread(os.path.join(dataset_awal, nama_file))

#     # Mengubah gambar menjadi grayscale (Ganti pake fungsi manual)
#     gambar_grayscale = cv.cvtColor(gambar_awal, cv.COLOR_BGR2GRAY)

#     # Melakukan resize (Ganti pake fungsi manual)
#     gambar_kecil = resize (gambar_grayscale)
#     # Melakukan thresholding (Ganti pake fungsi manual)
#     gambar_threshold = Threshold (gambar_kecil)
#     # Melakukan deteksi tepi metode Sobel (Ganti pake fungsi manual)
#     gambar_tepi = thinning (gambar_threshold)
#     # Melakukan normalisasi citra (Ganti pake fungsi manual)
#     gambar_normal = normalisasi (gambar_tepi)

#     cv.imwrite(os.path.join(dataset_thinning, nama_file), gambar_normal)

In [44]:
# Memuat data gambar
def load_data(data_dir):
    images = []
    labels = []

    for folder in os.listdir(data_dir):
        label = folder
        for image_filename in os.listdir(os.path.join(data_dir, folder)):
            image_path = os.path.join(data_dir, folder, image_filename)
            image = Image.open(image_path).resize((32, 32))
            image_data = np.array(image)
            images.append(image_data.flatten())
            labels.append(label)

    return np.array(images), np.array(labels)

In [45]:
X, y = load_data('dataset_thinning')

In [46]:
# Membagi data menjadi training dan testing (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [47]:
# Membuat model KNN
knn = KNeighborsClassifier(n_neighbors=5)

In [48]:
# Melatih model dengan data training
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [49]:
# Melakukan prediksi pada data testing
y_pred = knn.predict(X_test)

In [50]:
# Menghitung akurasi model
accuracy = np.mean(y_pred == y_test) * 100

In [51]:
# Menampilkan hasil akurasi
print("Akurasi model KNN:", accuracy, "%")

Akurasi model KNN: 36.19631901840491 %
